# Logistic Regression and TF SEAL Performance

## Logistic Regression

In this notebook, we will go through an example of logistic regression in TF SEAL, which will be a slightly modified version of the [logistic regression example on the TF SEAL GitHub](https://github.com/tf-encrypted/tf-seal/blob/master/examples/logistic_regression.py). Once again, we will be using the `customtf` conda environment made in the previous notebook. We also import the same libraries as before, with the addition of `time`, which will assist us in assessing the performance of TF SEAL operations.

In [4]:
import numpy as np
import tensorflow as tf
import tf_seal as tfs
import time

/home/dhairya/anaconda3/envs/customtf/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dhairya/anaconda3/envs/customtf/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dhairya/anaconda3/envs/customtf/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dhairya/anaconda

We also copy the sigmoid function approximation used by the example on the TF SEAL GitHub. An approximation is used here instead since the only two homomorphic operations we can work with are addition and multiplication. A function such as an exponential used in the sigmoid cannot be directly computed on the encrypted values. A potential get around is to use Taylor or Maclaurin Series of the function, which is a polynomial and would only require additions and multiplications. An issue with this could be that too many terms are required, and higher order polynomials requiring many multiplications would result in unacceptably slow speeds (why multiplications are slow will be explained further down). To illustrate how good a third order Taylor series compares to the third order polynomial used in the example, we can inspect their graphs compared to an actual sigmoid. In the graph below, the black curve is the actual sigmoid, the blue curve is the Taylor series, and the red curve is what is actually used.

![Comparison of Sigmoid Approximations](img/sigmoid_approx.png)

From a quick inspection, we can see that near 0, the Taylor series is an almost perfect approximation, but it diverges away from the sigmoid very quickly with such few terms. On the other hand, the red curve is approximately close to the sigmoid over a sizeable domain, even if it never models the behavior of the sigmoid closely in any small region. As we are already using approximations elsewhere in the calculations, approximating a sigmoid with the red curve is acceptable.

In [5]:
def sigmoid(x):
    # the following polynomial evaluation approximates by computing
    # 0.5 + 0.197x + -0.004x^3, the 0.0 term is ignored. This
    # approximation was borrowed from https://eprint.iacr.org/2018/462 Section 3.2 Fig 1
    coeffs = np.array([0.5, 0.197, 0.0, -0.004])
    return tfs.poly_eval(x, coeffs)

We can also further copy their logistic regression function for plain text values, which uses a more exact sigmoid function

In [6]:
def tf_log_reg(x, y):
    z = tf.matmul(x, y)
    return tf.sigmoid(z)

The encrypted and plaintext computations are done simply by using the previous functions. We do operations on encrypted functions exactly as we did in the previous notebook. We first get public and private keys, encrypt our data, and then do the computations. For the compuations, both the encrypted inputs, `a_plain`, as well as the public weights `b` are randomly generated, as we are only checking for the performance of our approximate sigmoid, rather than make any real predictions.

In [10]:
public_keys, secret_key = tfs.seal_key_gen(gen_relin=True, gen_galois=True)

# encrypted input -> tf_seal.Tensor
a_plain = np.random.normal(size=(2, 2)).astype(np.float32)
a = tfs.constant(a_plain, secret_key, public_keys)

# public weights
b = np.random.normal(size=(2, 2)).astype(np.float32)

c = tfs.matmul(a, b.transpose())
d = sigmoid(c)

# get answer from tensorflow to compare
tf_d = tf_log_reg(a_plain, b)

We modify the code for the TensorFlow session to incorporate the time it takes in order to do the regression through either method. We print both the times it takes for either method, and the ratio of the times to see how much slower TF SEAL is.

In [11]:
with tf.compat.v1.Session() as sess:
    print("\nTF SEAL Logistic Regression")

    time_ini1 = time.process_time()
    temp = sess.run(d)
    time_fin1 = time.process_time()
    print(temp)

    print("\nApproximately Equals\n")
    print("Tensorflow Logistic Regression")

    time_ini2 = time.process_time()
    temp = sess.run(tf_d)
    time_fin2 = time.process_time()
    print(temp)
    
    print("\nSEAL time:", int(1000000*(time_fin1 - time_ini1))/1000000, "s")
    print("Norm time:", int(1000000*(time_fin2 - time_ini2))/1000000, "s")
    print("\nSEAL is about", int((time_fin1 - time_ini1)/(time_fin2 - time_ini2)), "times slower than plaintext.\n")


TF SEAL Logistic Regression
[[0.66868596 0.98636614]
 [0.57309454 0.21389125]]

Approximately Equals

Tensorflow Logistic Regression
[[0.7046667  0.9932168 ]
 [0.59196186 0.17884232]]

SEAL time: 5.028836 s
Norm time: 0.001893 s

SEAL is about 2655 times slower than plaintext.



Comparing the values, we can see that our approximation is quite accurate, only differing by a few hundredths at most. This is a very good result considering our multiple approximations in arriving at the result. Another major result we see is the drastic difference in speed. The encrypted data is so large, computations on it need time on the order of seconds, while a *more difficult* computation with the sigmoid on plaintext data takes only milliseconds. A natural question to ask is where the bottleneck in the computations arises, so more efforts can be directed at tackling the aspect of the computations taking the longest. Until now, we have used the time library to evaluate the performance of the program using the time taken as a metric. This can only provide a limited amount of knowledge about the internals of the program. We will now switch over to using a profiler to get a deeper understanding of the bottlenecks of the program. 

## Setting up the Intel$^®$ VTune™ Profiler

The code profiler we will use here will be [Intel VTune](https://software.intel.com/content/www/us/en/develop/tools/vtune-profiler.html) on Ubuntu. After navigating to the previous link, you will find a "Choose & Download" button near the top right, leading you to [this page](https://software.intel.com/content/www/us/en/develop/tools/vtune-profiler/choose-download.html). Select the "Only Intel VTune Profiler" option, and then click "Download" under the "Download a free copy..." option. After filling in the information required, you will be able to download a tar file, named something like `vtune_profiler_20**_update*.tar.gz` with some numbers in place of the asterisks. The file is quite large, over 500 mb so it may take a while to download. Once it finishes downloading, navigate to where the file has been saved on your computer, and extract the contents to a folder (ideally with the same name `vtune_profiler_20**_update*`). After the extraction is finished, open the folder to find a bash script named `install.sh`. Open a terminal in this folder, and type

```
bash script-name-here.sh
```

to run the installation and follow the installation instructions. If no errors happened along the way, VTune should now be installed on your computer. At the end of the installation, *do not* immediately close the terminal. Your terminal should look like the image below:

![VTune Installation End Screen](img/vtune_end_screen.png)

Look at the section under "To set your environment variables", and save the command for bash users somewhere. In my case, the command is `source /home/dhairya/intel/vtune_profiler_2020.1.0.607630/vtune-vars.sh`. You will require this command before you use vtune in any new environment.

Now, open a new terminal in the directory where your logistic regression file is located and activate the `customtf` conda environment. In the terminal, first use the command saved from above, which in my case would be:

```
source /home/dhairya/intel/vtune_profiler_2020.1.0.607630/vtune-vars.sh
```

You will have to use this command *every* time you open a new `customtf` environment, such as after turning on your computer or after opening a new instance of the terminal.

For the next few steps you will require superuser access. After running the previous command, type these three commands into the terminal, and type the password when prompted.

```
echo 0|sudo tee /proc/sys/kernel/yama/ptrace_scope
echo 0|sudo tee /proc/sys/kernel/kptr_restrict
echo 0|sudo tee /proc/sys/kernel/perf_event_paranoid
```

If the files `/proc/sys/kernel/yama/ptrace_scope`, `/proc/sys/kernel/kptr_restrict`, and `proc/sys/kernel/perf_event_paranoid` already equal to 0, then these commands are not necessary. These values are stored in the RAM, and will be reset to 1 everytime you restart Ubuntu, unless you change certain settings (dangerous!).

## Using VTune™ to Profile the Logistic Regression

Once you have done this, you are ready to use VTune. To profile our logistic regression Python program, write

```
vtune -collect system-overview python [file_name].py
```

into the terminal, where `[file_name].py` is your name of the the Python file for logistic regression. This command tells VTune what information to collect, and for what executable. In this case, we are looking for information on the system as a whole while our python script is running. You may see a lot warnings while VTune collects information, but there is no need for concern. After VTune completes collecting the information, you will get a summary of the information at the bottom, such as what the major hotspots were (what processes were most taxing), and how long the program ran for. A more thorough analysis of the information collected by VTune will be saved in the a new directory named `r[3 digit number]so`. While you can get meaningful imformation about the bottlenecks of the program from the terminal output, there is a better way to see the information. Type the command

```
vtune-gui
```

in the terminal, which will open a GUI version of VTune. Go to the three lines in the top left corner, then to the "Open" option, and then click on "Result". Then navigate to the directory named `r[3 digit number]so`, as shown below, then select and open the file named `r[3 digit number]so.vtune`.

![VTune GUI](img/vtune_gui.png)

Opening the `.vtune` file will open a new tab in the program, which will look something like the following

![VTune Results GUI](img/vtune_results_gui.png)

From this, we can get some crucial information. At the top is the time it took for our logistic regression to execute, about 6 seconds. Below, in the section called "Top Hotspots", we can identify two SEAL functions that take a considerable amount of time, and are likely the bottlenecks of our program. Now, click on the "Platform" tab next to the "Summary" tab that is selected by default. At the bottom of the "Platform" tab, you can see a list of all of the functions that ran during the duration of execution, and how long each function took. 

![function time](img/function_time.png)

We can easily identify the SEAL functions that were being executed, and from those we can isolate ntt_negacyclic_harvey_lazy, and switch_key_inplace as the biggest bottlenecks on the execution.

## Understanding the Bottlenecks

To understand what operations the bottlenecks correspond to, we need to first understand a bit about homomorphic encryption. Firstly, we will explain what is happening in the second largest bottleneck. When encrypting data homomorphically, there are small error terms added to make it much harder to crack the ciphertext. When homomorphic operations are done, especially multiplications, these error terms compound and increase in size. If many such operations are done, then the error terms eventually increase in size until there is too much noise and the decrypted value is not equal, or even similar, to the correct value. In order to address this issue, the ciphertext is encrypted with a new key, and the old key is also encrypted with the new key. Then, the encrypted ciphertext is decrypted using the encrypted key. Since both encryption and decryption also only use additions and multiplications, they would maintain the homomorphic property. Also, by doing this, you can swap the encryption of the ciphertext from the old key to the new key, without someone ever finding the plaintext data. This operation is useful since it removes the error in the old ciphertext and resets it to a new small error in a new ciphertext. By continuously encrypting and decrypting, the error or noise in the ciphertext can be controlled and confined to be within an acceptable limit. The issue with this is that these operations are very costly, and that is why switch_key_inplace is the second largest bottleneck in the execution, as it is doing the operation described above.

Now, to understand what the largest bottleneck - ntt_negacyclic_harvey_lazy - represents, we have to begin with how the ciphertext is represented. Ciphertexts are stored as the coefficients of a polynomial, where both the degree of the polynomial and the coefficients are extremely large in magnitude. When multiplying ciphertexts, the computer is multiplying two polynomials, which can be a very costly operation for the large polynomials present in ciphertexts. But, if we store the polynomial coefficients in a vector, then the multiplication of the polynomials can be thought of as a cross-correlation between two vectors representing two polynomials. To illustrate this, let us look at the multiplication of two simple quadratics. 

$$
\begin{align*}
    p(x) &= x^2 + 2x + 3\\
    q(x) &= 4x^2 + 3x + 1\\
    r(x) &= p(x)q(x)\\
    &= x^2(4x^2 + 3x + 1) +2x(4x^2 + 3x + 1) + 3(4x^2 + 3x + 1)\\
    &= x^4(4\cdot1) + x^3(3\cdot1+4\cdot2) + x^2(1\cdot1+3\cdot2+4\cdot3)+x(1\cdot2+3\cdot3)+(1\cdot3)\\
    &= 4x^{4}+11x^{3}+19x^{2}+11x+3
\end{align*}
$$

Now let us define two vectors, $p$ and $q$, of the coefficients of the polynomials $p(x)$ and $q(x)$, where the $k$the term corresponds to the coefficient of $x^k$. Then, we can rewrite the second last line of the above expression as

$$
\begin{align*}
    r(x) &= x^4(4\cdot1) + x^3(3\cdot1+4\cdot2) + x^2(1\cdot1+3\cdot2+4\cdot3)+x(1\cdot2+3\cdot3)+(1\cdot3)\\
    r(x) &= x^4(q_2\cdot p_2) + x^3(q_1\cdot p_2+q_2\cdot p_1) + x^2(q_0\cdot p_2+q_1\cdot p_1+q_2\cdot p_0)+x(q_0\cdot p_1+q_1\cdot p_0)+(q_0\cdot p_0)\\
    r(x) &= \sum_{j=0}^\infty \sum_{i=0}^j p_i q_{j-i}x^j = \sum_{j=0}^\infty \sum_{i=0}^j q_i p_{j-i}x^j
\end{align*}
$$

where $a_k$ indicates the $k$th term of the vector a.

This expression simply states that the $j$th coefficient of the multiplied polynomial is a sum of the coefficients of the $x^j$ terms in the expansion of the multiplication. The $x^j$ terms would result when two terms, whose powers sum to $j$, are multiplied. This is shown in the coefficient $p_i q_{j-i}$, as those would be coefficients of terms which would multiply to give an $x^j$ term. If we further express this as an expression for the $j$th term of the vector $r$, which would simply be the coefficient of $x^j$, we get

$$
\begin{align*}
    r_j = \sum_{i=0}^j p_i q_{j-i} = \sum_{i=0}^j q_i p_{j-i}
\end{align*}
$$

In this form, we can more easily notice that this is simply the expression for a discrete convolution. Thus, we have found a way to reduce polynomial multiplication to a convolution of the coefficients, potentially simplifying the calculations for a computer. The main takeaway from all this is that if $r(x) = p(x)q(x)$, and $r$, $p$ and $q$ are the vectors of coefficients of their respective polynomials, $r = p*q$.

If we shift our focus to the Discrete Fourier Transform (DFT), we know the existence of a convolution theorem: $h = f*g \implies H = FG$, where $H$, $F$ and $G$ are the DFTs of the discrete finite vectors $h$, $f$ and $g$, respectively. When the vectors become sufficiently large in size, it is easier to do the DFT, multiply, and then do an inverse DFT to arrive at $h$, than it is to do the large amount of multiplications and additions to do a convolution. The Number Theoretic Transform (NTT), is a related transform to the DFT, which only involves integer additions and multiplications, as opposed to the much more complicated complex floating point operations required for a DFT. The NTT also transforms the vector representing a polynomial to another domain, where the convolution theorem still holds and the convolution can be replaced by a simple multiplication, and then transformed back to the regular domain. Although this is a vast improvement over direct multiplication, it is still quite slow on CPUs unoptimized for calculating NTTs, which is where the major bottleneck appears. Thus, the major bottleneck is ciphertext multiplication.

Note that even for extremely large polynomials, addition is not much slower, since the time taken for addition linearly increases with the number of terms in the polynomial, and can be neglected in considering the bottlenecks to the speed of execution.